# APIM ❤️ OpenAI

## Access Controlling lab
![flow](../../images/access-controlling.gif)

Playground to try the [OAuth 2.0 authorization feature](https://learn.microsoft.com/en-us/azure/api-management/api-management-authenticate-authorize-azure-openai#oauth-20-authorization-using-identity-provider) using identity provider to enable more fine-grained access to OpenAPI APIs by particular users or client.

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the App Registration in Microsoft Entra ID](#1)
- [2️⃣ Create the Azure Resource Group](#2)
- [3️⃣ Create deployment using 🦾 Bicep](#3)
- [4️⃣ Get the deployment outputs](#4)
- [5️⃣ Create a device flow to get the access token](#5)
- [6️⃣ Acquire the token and query the graph API](#6)
- [🧪 Test the API using a direct HTTP call](#requests)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 

In [44]:
import os
import json
import datetime
import requests

lab_prefix="ai-pevo6-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
#apim_resource_sku = "Basicv2"
apim_resource_sku = "Consumption"
openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "francecentral"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
#openai_resources = []
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ {"name": "openaimock1", "endpoint": f"https://{tool_prefix}openaimock1.azurewebsites.net"}, {"name": "openaimock2", "endpoint": f"https://{tool_prefix}openaimock2.azurewebsites.net"} ]

log_analytics_name = lab_prefix + "workspace"
app_insights_name = lab_prefix + 'insights'

app_registration_name = lab_prefix + "ai-gateway-openai-app"
print(mock_webapps)


[{'name': 'openaimock1', 'endpoint': 'https://st-pevo1-openaimock1.azurewebsites.net'}, {'name': 'openaimock2', 'endpoint': 'https://st-pevo1-openaimock2.azurewebsites.net'}]


<a id='1'></a>
### 1️⃣ Create the App Registration in Microsoft Entra ID
The following command creates a client application registration

In [45]:
cmd_stdout = ! az account show --query homeTenantId --output tsv
tenant_id = cmd_stdout.n

cmd_stdout = ! az ad app create --display-name {app_registration_name} --query appId --is-fallback-public-client true --output tsv
client_id = cmd_stdout.n
print("✅ App registration: ",  app_registration_name, " created. Client_id: ", client_id, " ⌚ ", datetime.datetime.now().time())


✅ App registration:  ai-pevo5-ai-gateway-openai-app  created. Client_id:  7e489c2c-f24a-42ea-8a38-5298e9a4f6ec  ⌚  14:25:15.038317


<a id='2'></a>
### 2️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [46]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  ai-pevo5-lab-access-controlling  created ⌚  14:25:20.120106


<a id='3'></a>
### 3️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [47]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")
elif len(mock_webapps) > 0:
    backend_id = mock_backend_pool if len(mock_backend_pool) > 1 else mock_webapps[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id).replace("{aad-client-application-id}", client_id).replace("{aad-tenant-id}", tenant_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


{
  "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ai-pevo5-lab-access-controlling/providers/Microsoft.Resources/deployments/access-controlling",
  "location": null,
  "name": "access-controlling",
  "properties": {
    "correlationId": "2c62e88b-28db-42a8-b527-24f9867d717e",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ai-pevo5-lab-access-controlling/providers/Microsoft.ApiManagement/service/apim-qt2wygzdvlsbo",
            "resourceGroup": "ai-pevo5-lab-access-controlling",
            "resourceName": "apim-qt2wygzdvlsbo",
            "resourceType": "Microsoft.ApiManagement/service"
          }
        ],
        "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ai-pevo5-lab-access-controlling/providers/Microsoft.ApiManagement/service/apim-qt2wygzdvlsbo/apis/openai",
        "resourceGroup": "ai-pevo5-l


c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\labs\access-controlling\main.bicep(299,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]




867

<a id='4'></a>
### 4️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [48]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimServiceId.value -o tsv
apim_service_id = deployment_stdout.n
print("👉🏻 APIM Service Id: ", apim_service_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)

👉🏻 APIM Service Id:  /subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ai-pevo5-lab-access-controlling/providers/Microsoft.ApiManagement/service/apim-qt2wygzdvlsbo
👉🏻 API Gateway URL:  https://apim-qt2wygzdvlsbo.azure-api.net
👉🏻 Workspace ID:  bdc6cebd-bf83-41b4-ae83-2f46d0227bcf
👉🏻 App ID:  5d4a2ac4-01ff-4362-8349-e0064e5645df


<a id='5'></a>
### 5️⃣ Create a device flow to get the access token

Notes for fine grained authorization:
- The APIM [JWT validation policy](https://learn.microsoft.com/en-us/azure/api-management/validate-azure-ad-token-policy) can check for specific claims (that needs to exist in the token) and apply fine-grained authorization.
- Group claims is a typical method. You can use this approach to drive authorization. However, when the user is a member of too many groups, the `groups` will be excluded from the token due to limitations in token size.
- An alternative is to configure app role definitions and assign users/groups to app roles. This Zero Trust developer best practice improves flexibility and control while increasing application security with least privilege. [Learn more](https://learn.microsoft.com/en-us/security/zero-trust/develop/configure-tokens-group-claims-app-roles).
- To obtain the `roles` claim, navigate to the "Expose an API" section of the App Registration. Add the Application ID URI and a scope. Then, copy the full scope (app://<id>/scope) and add it to the scopes array below.
- Navigate to the "App Roles" blade and create an App Role (ex: OpenAI.ChatCompletion) for Users/Groups members. Then assign the testing user or group to the App Role.   
- After logging in, use https://jwt.io/ to decode the `access_token` variable and verify that the `roles` are being sent.
- With the above configuration, you can add the following fragment to the APIM policy to verify that the user belongs to a specific App Role:
```
            <required-claims>
                <claim name="roles" match="any">
                    <value>OpenAI.ChatCompletion</value>
                </claim>
            </required-claims>
```



In [49]:
import json
import logging

import requests
import msal

app = msal.PublicClientApplication(
    client_id, authority="https://login.microsoftonline.com/" + tenant_id)

flow = app.initiate_device_flow(scopes=["User.Read"])
if "user_code" not in flow:
    raise ValueError(
        "Fail to create device flow. Err: %s" % json.dumps(flow, indent=4))

print(flow["message"])



To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code G8A727P89 to authenticate.


<a id='6'></a>
### 6️⃣ Acquire the token and query the graph API

In [50]:
result = app.acquire_token_by_device_flow(flow)

if "access_token" in result:
    access_token = result['access_token']
    # Calling graph using the access token
    graph_data = requests.get(  # Use token to call downstream service
        "https://graph.microsoft.com/v1.0/me",
        headers={'Authorization': 'Bearer ' + access_token},).json()
    print("Graph API call result: %s" % json.dumps(graph_data, indent=2))
    # print(access_token) # Use a tool like https://jwt.io/ to decode the access token and see its contents
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))  # You may need this when reporting a bug

Graph API call result: {
  "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users/$entity",
  "businessPhones": [],
  "displayName": "Per Vorm - P",
  "givenName": "Per - P",
  "jobTitle": null,
  "mail": null,
  "mobilePhone": null,
  "officeLocation": null,
  "preferredLanguage": null,
  "surname": "Vorm",
  "userPrincipalName": "pevo_sdc.dk#EXT#@pevosdc.onmicrosoft.com",
  "id": "233aca6e-cd0c-4453-a4bb-3f7d47227789"
}


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [52]:
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}
response = requests.post(url, headers = {'api-key':apim_subscription_key, 'Authorization': 'Bearer ' + access_token}, json = messages)
print("status code: ", response.status_code)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("response: ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)


status code:  200
response:  Oh, I'm sorry, I don't do time. That's way above my pay grade. You might want to try looking at your phone or maybe even a clock. Just a suggestion.


<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK
OpenAPI provides a widely used [Python library](https://github.com/openai/openai-python). The library includes type definitions for all request params and response fields. The goal of this test is to assert that APIM can seamlessly proxy requests to OpenAI without disrupting its functionality.
- Note: run ```pip install openai``` in a terminal before executing this step.

In [54]:

from openai import AzureOpenAI
messages=[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]
client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version        
)
response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"Authorization": "Bearer " + access_token})
print(response.choices[0].message.content)


Oh, I'm sorry, I must have left my psychic abilities at home today. I can't exactly tell you the time without some context. Maybe you could try looking at the clock on your wall or your fancy watch? Just a thought.


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.